In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import Libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import numpy as np

# Part 1 - Exploratory Analysis

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/Mineral Resource Management/Mines_and_Mineral_Resources.csv')

In [ ]:
# Display the first few rows of the dataframe
print(data.head())

In [ ]:
# Summary statistics
print(data.describe())

In [ ]:
# Data types of columns
print(data.dtypes)

In [ ]:
# Check for missing values
print(data.isnull().sum())

In [ ]:
# Histogram for AREA_ column
plt.hist(data['AREA_'], bins=20, color='skyblue', edgecolor='black')
plt.title('Distribution of Mine Area')
plt.xlabel('Area')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Bar plot for categorical features
plt.figure(figsize=(10, 6))
data['FEATTYPE'].value_counts().plot(kind='bar', color='lightcoral')
plt.title('Frequency of Feature Types')
plt.xlabel('Feature Type')
plt.ylabel('Frequency')
plt.show()

# Part 2 - Data Preprocessing

In [ ]:
selected_features = ['FEATTYPE', 'SECCLASS', 'STATE', 'ZIP', 'COUNTY', 'NAICSDESCR', 'X', 'Y']
target_variable = 'MINE_TYPE'

In [ ]:
data_subset = data[selected_features + [target_variable]].copy()  # Use .copy() to create a copy of the DataFrame

In [ ]:
data_subset['MINE_TYPE'] = data_subset['MINE_TYPE'].astype(str)

## Encode Categorical Variables

In [ ]:
label_encoders = {}
for feature in selected_features:
    if data_subset[feature].dtype == 'object':
        label_encoders[feature] = LabelEncoder()
        data_subset[feature] = label_encoders[feature].fit_transform(data_subset[feature])

In [ ]:
# Split the data into features (X) and target variable (y)
X = data_subset.drop(columns=[target_variable])
y = data_subset[target_variable]

In [ ]:
label_encoder_y = LabelEncoder()
y_encoded = label_encoder_y.fit_transform(y)

## Split the Data

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Part 3 - Train the Model

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)
print("Model trained.")

# Part 4 - Make Predictions

In [ ]:
# Make predictions on the testing set
y_pred = model.predict(X_test)

In [ ]:
# Decode the predicted labels
y_pred_decoded = label_encoder_y.inverse_transform(y_pred)

# Part 5 - Evaluate Model Performance

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9566666666666667


In [ ]:
# Classification report
print(classification_report(y_test, y_pred, target_names=label_encoder_y.classes_))

In [ ]:
# Analyze feature importance
feature_importance = model.feature_importances_
print("Feature Importance:")
for i, feature in enumerate(X.columns):
    print(feature, ":", feature_importance[i])

Feature Importance:
FEATTYPE : 0.0
SECCLASS : 0.0
STATE : 0.06455076
ZIP : 0.041372996
COUNTY : 0.03131092
NAICSDESCR : 0.8015152
X : 0.038793158
Y : 0.022457026
